## ✅ Roadmap del Proyecto de Web Scraping y Carga a Base de Datos

Este proyecto tiene como objetivo scrapear libros desde una web pública, obtener información adicional mediante una API externa y almacenar los datos en una base de datos relacional para su posterior análisis.

### 🛠️ Etapas del desarrollo

- [ ] **1. Web Scraping**
  - [ ] Scrapear todos los géneros desde la página principal.
  - [ ] Iterar sobre cada género para obtener todos los libros listados.
  - [ ] Extraer título, precio, stock, rating y link a la página del libro.
  - [ ] Extraer información detallada desde la página de cada libro (si es necesario).
  - [ ] Consultar la API de Google Books para obtener el autor y más detalles (usando el título).
  - [ ] Guardar los datos obtenidos en archivos `.csv` separados:
    - [ ] Libros (`libros.csv`)
    - [ ] Autores (`autores.csv`)
    - [ ] Géneros (`generos.csv`)

- [ ] **2. Validación**
  - [ ] Verificar la integridad y limpieza de los datos descargados.
  - [ ] Eliminar duplicados y manejar valores nulos si los hay.

- [ ] **3. Diseño de Base de Datos Relacional**
  - [ ] Crear script DDL:
    - [ ] Crear la base de datos (si no existe).
    - [ ] Crear las tablas `libros`, `autores`, `generos`, y tablas intermedias para relaciones N:N si aplica.
  - [ ] Crear diagrama UML/ER para visualizar relaciones.

- [ ] **4. Inserción de Datos**
  - [ ] Crear script DML para insertar los datos desde los CSV a la base de datos (usando Python + `psycopg2` o `SQLAlchemy` para PostgreSQL).
  - [ ] Comprobar inserciones correctas mediante queries de prueba.

- [ ] **5. Consultas y Análisis**
  - [ ] Escribir consultas SQL para:
    - [ ] Obtener todos los libros por género.
    - [ ] Buscar libros por autor.
    - [ ] Calcular estadísticas como precio promedio por género o autor.
  - [ ] (Opcional) Crear una vista para simplificar reportes.

---

> 📌 **Nota:** La base de datos será inicialmente verificada con SQLite por simplicidad, luego se adaptará a PostgreSQL para una implementación más robusta con PgAdmin.


In [1]:
!pip install BeautifulSoup4
!pip install sqlalchemy psycopg2-binary pandas


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached sqlalchemy-2.0.42-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached psycopg2_binary-2.9.10-cp313-cp313-win_amd64.whl.metadata (4.8 kB)
  Using cached greenlet-3.2.3-cp313-cp313-win_amd64.whl.metadata (4.2 kB)
Using cached sqlalchemy-2.0.42-cp313-cp313-win_amd64.whl (2.1 MB)
Using cached psycopg2_binary-2.9.10-cp313-cp313-win_amd64.whl (2.6 MB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.0 MB 9.7 MB/s eta 0:00:02
   ----------------------------- ---------- 8.1/11.0 MB 24.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 28.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 16.3 MB/s eta 0:00:00
Using cached greenlet-3.2.3-cp313-cp313-win_amd64.whl (297 kB)
   -------------------------

In [ ]:
# import web grabbing client and
# HTML parser
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

# variable to store website link as string
myurl = 'http://books.toscrape.com/index.html'

# grab website and store in variable uclient
uClient = uReq(myurl)

# read and close HTML para no sobrecargar de pedidos
page_html = uClient.read()
uClient.close()

# call BeautifulSoup for parsing
page_soup = soup(page_html, "html.parser")

# grabs all the products under list tag
bookshelf = page_soup.find(
    "ul", {"class": "nav nav-list"})
links = bookshelf.findAll("a")

lista_libros= []
i=0
for link in links:
    if i != 0:
        # collect link of all genders
        a = link.get("href")
        lista_libros.append(a)
    i=i+1 # just to ignore the first

for genero in lista_libros:
    uGender = uReq(genero)
    pagina_libro = uGender.read()
    uGender.close()

    gender_soup =(pagina_libro, "html.parser")
    libro = gender_soup.findAll("li", {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    direccion = libro.findAll("a")
    for direc in direccion:
        direcciones = direc.get("href")










        print(direcciones, "\n")


C:\Users\soporte\AppData\Local\Temp\ipykernel_22944\1569491476.py:22: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  links = bookshelf.findAll("a")


ValueError: unknown url type: 'catalogue/category/books/travel_2/index.html'